In [376]:
!pip install pyspark

In [377]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, regexp_replace
from pyspark import SparkConf,SparkContext
from pyspark.sql.functions import split, concat_ws

In [378]:
data = pd.read_csv("dataset15k.csv")

In [379]:
data = data.drop("Summary", axis=1)
data = data.drop("img_m", axis=1)
data = data.drop("img_s", axis=1)
data = data.drop("img_l", axis=1)
data = data.drop("Category", axis=1)
data['isbn'] = pd.to_numeric(data['isbn'], errors='coerce',downcast='integer')
data = data.dropna()
data['isbn'] = data['isbn'].astype(int)
data['user_id'] = data["user_id"].astype(str).str[:4]
data['isbn'] = data["isbn"].astype(str).str[:9]
new_df = data.drop("location", axis=1)

In [380]:
new_df.to_csv("newdataset.csv", index=False)

In [381]:
d = pd.read_csv("newdataset.csv")

In [382]:
d.head()

,Unnamed: 0,user_id,age,isbn,rating,book_title,book_author,year_of_publication,publisher,Language,city,state,country
0,0,2,18.0000,195153448,0,Classical Mythology,Mark P. O. Morford,2002.0,Oxford University Press,en,stockton,california,usa
1,1,8,34.7439,2005018,5,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,en,timmins,ontario,canada
2,2,1140,49.0000,2005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,en,ottawa,ontario,canada
3,4,4138,34.7439,2005018,0,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,en,sudbury,ontario,canada
4,5,6754,30.0000,2005018,8,Clara Callan,Richard Bruce Wright,2001.0,HarperFlamingo Canada,en,toronto,ontario,canada


In [383]:
conf = SparkConf().setAppName("191805056_EsmanurDeli")
sc = SparkContext.getOrCreate(conf=conf)

ConnectionRefusedError: [Errno 61] Connection refused

In [372]:
dataset = sc.textFile("newdataset.csv")

ConnectionRefusedError: [Errno 61] Connection refused

In [373]:
header = dataset.first()

ConnectionRefusedError: [Errno 61] Connection refused

In [374]:
data = dataset.filter(lambda line: line != header)

ConnectionRefusedError: [Errno 61] Connection refused

In [375]:
num_books = data.count()
print("Number of Books: ", num_books)

ConnectionRefusedError: [Errno 61] Connection refused

Transform the data by splitting each record into fields.

In [351]:
book_fields = data.map(lambda lines: lines.split(","))

In [352]:
book_fields.first()

['0',
 '2',
 '18.0',
 '195153448',
 '0',
 'Classical Mythology',
 'Mark P. O. Morford',
 '2002.0',
 'Oxford University Press',
 'en',
 'stockton',
 'california',
 'usa']

Gerekli yerleri nümerik değere çevirme

In [353]:
isbn = book_fields.map(lambda fields:fields[3])

In [354]:
isbn.map(lambda x: int(x) if x.isdigit() else None).filter(lambda x: x is not None)

PythonRDD[250] at RDD at PythonRDD.scala:53

In [355]:
user_id = book_fields.map(lambda fields:fields[1])

In [356]:
user_id.map(lambda x: int(x) if x.isdigit() else None).filter(lambda x: x is not None)

PythonRDD[251] at RDD at PythonRDD.scala:53

In [357]:
rating = book_fields.map(lambda fields:fields[4])

In [358]:
rating.map(lambda x: int(x) if x.isdigit() else None).filter(lambda x: x is not None)

PythonRDD[252] at RDD at PythonRDD.scala:53

In [359]:
user_id_filtered.take(3)

[2, 8, 1140]

In [360]:
rating.take(3)

['0', '5', '0']

Build recommendation engine using collaborative filtering.

In [361]:
from pyspark.mllib. recommendation import ALS # Alternating Least Squares 
from pyspark.mllib.recommendation import Rating
ratings = book_fields.map (lambda x: Rating(int(x[1]), int(x[3]), float(x[4])))

In [362]:
ratings.take(5)

[Rating(user=2, product=195153448, rating=0.0),
 Rating(user=8, product=2005018, rating=5.0),
 Rating(user=1140, product=2005018, rating=0.0),
 Rating(user=4138, product=2005018, rating=0.0),
 Rating(user=6754, product=2005018, rating=8.0)]

Use %70 of dataset for training, rest of them for testing.

In [363]:
training_data, test_data = ratings.randomSplit([0.7, 0.3])

In [364]:
training_data.take(10)

[Rating(user=8, product=2005018, rating=5.0),
 Rating(user=4138, product=2005018, rating=0.0),
 Rating(user=6754, product=2005018, rating=8.0),
 Rating(user=8552, product=2005018, rating=0.0),
 Rating(user=2002, product=2005018, rating=8.0),
 Rating(user=2190, product=2005018, rating=7.0),
 Rating(user=2633, product=2005018, rating=6.0),
 Rating(user=2954, product=60973129, rating=8.0),
 Rating(user=1528, product=60973129, rating=7.0),
 Rating(user=8, product=374157065, rating=0.0)]

In [365]:
training_data = training_data.map(lambda x: (x[0], x[1], int(x[2])))

In [366]:
training_data.take(10)

[(8, 2005018, 5),
 (4138, 2005018, 0),
 (6754, 2005018, 8),
 (8552, 2005018, 0),
 (2002, 2005018, 8),
 (2190, 2005018, 7),
 (2633, 2005018, 6),
 (2954, 60973129, 8),
 (1528, 60973129, 7),
 (8, 374157065, 0)]

Use ALS (Alternating Least Squares) for training recommendation model with last 4 digit
of your student number as a “seed”. Also change the parameters of ALS re-run the
algorithm for parameters “rank” (10, 50, 200), “iteration” (10, 50, 200) and “lambda”
(0.01, 0.1). 

In [369]:
from pyspark.mllib.evaluation import RegressionMetrics

seed = 5056
rank_values = [10, 50, 200]
iteration_values = [10, 50, 200]
lambda_values = [0.01, 0.1]

In [370]:
results = []

for rank in rank_values:
    for iteration in iteration_values:
        for lmbda in lambda_values:
            model = ALS.train(
                training_data,
                rank=rank,
                iterations=iteration,
                lambda_=lmbda,
                seed=seed
            )

            # Test verisi üzerinde tahminler yapma
            test_user_product = test_data.map(lambda x: (x[0], x[1]))
            predictions = model.predictAll(test_user_product).map(lambda x: ((x[0], x[1]), x[2]))
            rates_and_preds = test_data.map(lambda x: ((x[0], x[1]), x[2])).join(predictions)
            mse = rates_and_preds.map(lambda x: (x[1][0] - x[1][1])**2).mean()
            rmse = mse**0.5

            result = {
                'Rank': rank,
                'Iterations': iteration,
                'Lambda': lmbda,
                'MSE': mse,
                'RMSE': rmse
            }
            results.append(result)

            print("Model Parameters: Rank={}, Iterations={}, Lambda={}".format(rank, iteration, lmbda))
            print("MSE:", mse)
            print("RMSE:", rmse)
            print("-------------------------------")


ConnectionRefusedError: [Errno 61] Connection refused